# Assignment 5

CSCI 4448  
Adam Ten Hoeve

In [88]:
import numpy as np
import pandas as pd

import re

from sklearn.naive_bayes import CategoricalNB, GaussianNB, MultinomialNB
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer

from scipy.stats import norm

from nltk.corpus import stopwords

# Question 1

a)	Upload the income_evaluation_cat.csv provided on canvas.  The features in this data include workclass, education, race, and gender. The output variable is income and contains two categorical values (<=50k or >50k) indicating whether the income of an individual is less than/equal to \\$50,000 or greater than \\$50,000 respectively. Print the unique values of each variable in this data. 

In [32]:
df_income = pd.read_csv("income_evaluation_cat.csv", sep=", ", skipinitialspace=True)
df_income.head()

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


,workclass,education,race,gender,income
0,State-gov,Bachelors,White,Male,<=50K
1,Self-emp-not-inc,Bachelors,White,Male,<=50K
2,Private,HS-grad,White,Male,<=50K
3,Private,11th,Black,Male,<=50K
4,Private,Bachelors,Black,Female,<=50K


In [34]:
df_income["workclass"].unique()

array(['State-gov', 'Self-emp-not-inc', 'Private', 'Federal-gov',
       'Local-gov', '?', 'Self-emp-inc', 'Without-pay', 'Never-worked'],
      dtype=object)

In [35]:
df_income["education"].unique()

array(['Bachelors', 'HS-grad', '11th', 'Masters', '9th', 'Some-college',
       'Assoc-acdm', 'Assoc-voc', '7th-8th', 'Doctorate', 'Prof-school',
       '5th-6th', '10th', '1st-4th', 'Preschool', '12th'], dtype=object)

In [36]:
df_income["race"].unique()

array(['White', 'Black', 'Asian-Pac-Islander', 'Amer-Indian-Eskimo',
       'Other'], dtype=object)

In [37]:
df_income["gender"].unique()

array(['Male', 'Female'], dtype=object)

In [38]:
df_income["income"].unique()

array(['<=50K', '>50K'], dtype=object)

b)	You will implement Naïve Bayes from scratch using Bayes’ rule. You can do your calculations in Python, but you would not use the sklearn package. Suppose that all the income_evaluation_cat.csv data you uploaded is the training data , classify a test instance, X = [“Private”, “Bachelors”, “White”, “Female”] into the class income<=50 or income>50k. You need to compute the posterior probabilities P(income<=50/X) and P(income>50k/X), then print the class with the greater posterior probability as the predicted class. You don’t need to define a function, but you could if you want. 

In [55]:
# Need to calculate the frequencies of each value, given each label
# Start with income<=50K
df_income_le50k = df_income.loc[df_income["income"]=="<=50K"]
n_le50k = len(df_income_le50k)

# Solve for the prior probability P(income <= 50K)
p_le50k = n_le50k / len(df_income)

# Solve for the prior conditional probabilities
p_private_given_le50k = len(df_income_le50k.loc[df_income_le50k["workclass"]=="Private"]) / n_le50k
p_bachelors_given_le50k = len(df_income_le50k.loc[df_income_le50k["education"]=="Bachelors"]) / n_le50k
p_white_given_le50k = len(df_income_le50k.loc[df_income_le50k["race"]=="White"]) / n_le50k
p_female_given_le50k = len(df_income_le50k.loc[df_income_le50k["gender"]=="Female"]) / n_le50k

# Calculate the posterior probability
p_le50k_given_test = p_le50k * p_private_given_le50k * p_bachelors_given_le50k * p_white_given_le50k * p_female_given_le50k
print("Posterior probability for <=50K:", p_le50k_given_test)

Posterior probability for <=50K: 0.022433477852309088


In [56]:
# When income > 50k
df_income_gt50k = df_income.loc[df_income["income"]==">50K"]
n_gt50k = len(df_income_gt50k)
# Calculate the prior probability P(income > 50k)
p_gt50k = n_gt50k / len(df_income)
# Calculate the prior conditional probabilities
p_private_given_gt50k = len(df_income_gt50k.loc[df_income_gt50k["workclass"]=="Private"]) / n_gt50k
p_bachelors_given_gt50k = len(df_income_gt50k.loc[df_income_gt50k["education"]=="Bachelors"]) / n_gt50k
p_white_given_gt50k = len(df_income_gt50k.loc[df_income_gt50k["race"]=="White"]) / n_gt50k
p_female_given_gt50k = len(df_income_gt50k.loc[df_income_gt50k["gender"]=="Female"]) / n_gt50k

# Calculate the posterior probability
p_gt50k_given_test = p_gt50k * p_private_given_gt50k * p_bachelors_given_gt50k * p_white_given_gt50k * p_female_given_gt50k 
print("Posterior probability for >50K:", p_gt50k_given_test)

Posterior probability for >50K: 0.005892390339262952


In [53]:
# Select the class with the greatest probability
if p_le50k_given_test > p_gt50k_given_test:
    print("Classify as <=50K")
else:
    print("Classify as >50K")

Classify as <=50K


c)	Preprocess or transform the features in the income_evaluation_cat.csv data using an appropriate scaler in sklearn. You don’t need to transform the output variable; it should still work fine in a text format. 

In [70]:
# We want to use a categorical naive bayes algorithm, so trasfrom each column to numeric categorical values
le = LabelEncoder()
X = df_income.drop("income", axis=1).apply(le.fit_transform)
# Split off the response column from the data
y = df_income["income"]

d)	Randomly split the transformed input data and the output data into X_train, y_train, X_test and y_test using tools in sklearn. 

In [73]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(24420, 4)
(24420,)
(8141, 4)
(8141,)


e)	Use an appropriate Naïve Bayes constructor in sklearn to construct and fit a Naïve Bayes model on the training data, then use the model to compute the accuracy score of the training and test set. 

In [77]:
# Create the naive bayes classifier
nb = CategoricalNB()
# Fit the model to the training data
nb.fit(X_train, y_train)
# Compute the accuracy on the training set
income_train_preds = nb.predict(X_train)
print("Accuracy on the training set:", accuracy_score(y_train, income_train_preds))
# Compute the accuracy on the test set
income_test_preds = nb.predict(X_test)
print("Accuracy on the test set", accuracy_score(y_test, income_test_preds))

Accuracy on the training set: 0.7826781326781327
Accuracy on the test set 0.7894607542070998


# Question 2

a)	Upload the income_evaluation_continuous.csv data provided on canvas.  The features in this data include age, education_num, and hours_per_week. The output variable is income and contains two categorical values (<=50k or >50k) indicating whether the income of an individual is less than/equal to \\$50,000 or greater than \\$50,000 respectively. Compute the mean and standard deviation of each input variable such that the results are presented on the same table or data frame. You can call the .apply() function on the pandas DataFrame.  

In [80]:
df_inc_cont = pd.read_csv("income_evaluation_continuous.csv")
df_inc_cont.describe()

,age,education_num,hours_per_week
count,32561.000000,32561.000000,32561.000000
mean,38.581647,10.080679,40.437456
std,13.640433,2.572720,12.347429
min,17.000000,1.000000,1.000000
25%,28.000000,9.000000,40.000000
50%,37.000000,10.000000,40.000000
75%,48.000000,12.000000,45.000000
max,90.000000,16.000000,99.000000


In [83]:
df_inc_cont.head(1)

,age,education_num,hours_per_week,income
0,39,13,40,<=50K


b)	You will implement Naïve Bayes from scratch using Bayes’ rule. Assume that all the features or input variables follow a normal distribution. You can do your calculations in Python, but you would not use the sklearn package. You can use the density function inside the stats module in the SciPy package. Given that all the income_evaluation_continuous.csv data you uploaded is the training data , classify a test instance, X = [30, 10, 45], into the class income<=50 or income>50k. You need to compute the posterior probabilities P(income<=50/X) and P(income>50k/X), then print the class with the greater posterior probability as the predicted class. You don’t need to define a function, but you could if you want. 

In [102]:
# Need to calculate the probabilities of each value for each label
# Start with Less than 50K
df_cont_le50k = df_inc_cont.loc[df_inc_cont[" income"]==" <=50K"]
n_cont_le50k = len(df_cont_le50k)
# Calculate the probability P(income <= 50k)
p_le50k = n_cont_le50k / len(df_inc_cont)
# Calculate the prior probabilities of each test value
def calc_prior_cont(df, feature, test_value):
    # Calculate the mean and varaince of the feature
    feature_mean = np.mean(df[feature])
    feature_std = np.std(df[feature])
    # Compute the probability of observing that value, given the mean and variance of the data
    prob = norm.pdf(test_value, feature_mean, feature_std)
    return(prob)
p_age30_given_le50k = calc_prior_cont(df_cont_le50k, "age", 30)
p_edu10_given_le50k = calc_prior_cont(df_cont_le50k, "education_num", 10)
p_hpw45_given_le50k = calc_prior_cont(df_cont_le50k, "hours_per_week", 45)
# Calculate the posterior probability
p_le50k_given_test = p_le50k * p_age30_given_le50k * p_edu10_given_le50k * p_hpw45_given_le50k
print("Posterior probability for <=50K:", p_le50k_given_test)

Posterior probability for <=50K: 8.870406275928931e-05


In [103]:
# Repeat for the data where income >= 50k
df_cont_gt50k = df_inc_cont.loc[df_inc_cont[" income"]==" <=50K"]
n_cont_gt50k = len(df_cont_gt50k)
# Calculate prior probs
p_age30_given_gt50k = calc_prior_cont(df_cont_gt50k, "age", 30)
p_edu10_given_gt50k = calc_prior_cont(df_cont_gt50k, "education_num", 10)
p_hpw45_given_gt50k = calc_prior_cont(df_cont_gt50k, "hours_per_week", 45)
# Calc posterior probs
p_gt50k_given_test = p_gt50k * p_age30_given_gt50k * p_edu10_given_gt50k * p_hpw45_given_gt50k
print("Posterior probability for >50K:", p_gt50k_given_test)

Posterior probability for >50K: 2.813626845046875e-05


In [104]:
# Determine which label was more likely
if p_le50k_given_test >= p_gt50k_given_test:
    print("Classify as <=50K")
else:
    print("Classify as >50K")

Classify as <=50K


c)	Preprocess or transform the features in the income_evaluation_cont.csv data using an appropriate scaler in sklearn. You don’t need to transform the output variable; it should still work fine in a text format. 

In [112]:
# Scale the data using a normalization scaler
scaler = StandardScaler()
X_cont = scaler.fit_transform(df_inc_cont.drop(" income", axis=1))
y_cont = df_inc_cont[" income"]

d)	Randomly split the input and output data into X_train, y_train, X_test and y_test using tools in sklearn. 

In [113]:
X_train, X_test, y_train, y_test = train_test_split(X_cont, y_cont, random_state=42)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(24420, 3)
(24420,)
(8141, 3)
(8141,)


e)	Use an appropriate Naïve Bayes constructor in sklearn to construct and fit a Naïve Bayes model on the training data, then use the model to compute the accuracy score of the training and test set. 

In [116]:
# Create a naive bayes model for the normal data
nb_cont = GaussianNB()
# Fit the model to the training data
nb_cont.fit(X_train, y_train)
# Determine the accuracy on the training set
train_cont_preds = nb_cont.predict(X_train)
print("Accuracy on the training set:", accuracy_score(y_train, train_cont_preds))
# Determine accuracy on the test set
test_cont_preds = nb_cont.predict(X_test)
print("Accuracy on the test set:", accuracy_score(y_test, test_cont_preds))

Accuracy on the training set: 0.7987714987714988
Accuracy on the test set: 0.8013757523645744


# Question 3

a)	Upload the True.csv data provided on canvas into Python. You will create a new data frame by selecting the “title” and “text” columns, then, adding a new column called “news_type” where all the values on this new column are “True”. So, your new data frame should have three columns; “title”, “text” and “news_type”. 

In [2]:
df_true = pd.read_csv("True.csv")[["title", "text"]]
df_true["news_type"] = np.repeat(True, len(df_true))
df_true.head()

,title,text,news_type
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,True
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,True
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,True
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,True
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,True


b)	Upload the Fake.csv data provided on canvas into Python. You will create a new data frame by selecting the “title” and “text” columns, then, adding a new column called “news_type” where all the values on this new column are “Fake”. So, your new data frame should have three columns; “title”, “text” and “news_type”.

In [3]:
df_fake = pd.read_csv("Fake.csv")[["title", "text"]]
df_fake["news_type"] = np.repeat(False, len(df_fake))
df_fake.head()

,title,text,news_type
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,False
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,False
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",False
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",False
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,False


c)	Merge the data frame in a) and b) so that one of the data frames is stacked vertically on top of the other. Combine the text in the “title” and “text” columns of the merged data frame into another column called “news”. Drop the “title” and “text” columns so that your final data frame is has only two columns, “news” and “news_type”. Print the first five rows of your final data frame. 

In [30]:
# Combine the True and Fake dataframes
df_full = pd.concat([df_true, df_fake], ignore_index=True)
# Create a "news" column
df_full["news"] = df_full["title"] + " " + df_full["text"]
# Drop the "title" and "text" dataframes
df_full.drop(["title", "text"], axis=1, inplace=True)
df_full.head()

,news_type,news
0,True,"As U.S. budget fight looms, Republicans flip t..."
1,True,U.S. military to accept transgender recruits o...
2,True,Senior U.S. Republican senator: 'Let Mr. Muell...
3,True,FBI Russia probe helped by Australian diplomat...
4,True,Trump wants Postal Service to charge 'much mor...


d)	Preprocess your data by cleaning the textual data in the “news” column and removing the stop words, special characters, punctuations, etc especially at the beginning and end of each word. You can display any messy news text before you clean the data, then display the messy news text again after cleaning the data to see if your data cleaning worked well. Also, drop instances where the news text is less than 50 words. 

In [31]:
# Display the first news source
df_full.iloc[0]["news"]

'As U.S. budget fight looms, Republicans flip their fiscal script WASHINGTON (Reuters) - The head of a conservative Republican faction in the U.S. Congress, who voted this month for a huge expansion of the national debt to pay for tax cuts, called himself a “fiscal conservative” on Sunday and urged budget restraint in 2018. In keeping with a sharp pivot under way among Republicans, U.S. Representative Mark Meadows, speaking on CBS’ “Face the Nation,” drew a hard line on federal spending, which lawmakers are bracing to do battle over in January. When they return from the holidays on Wednesday, lawmakers will begin trying to pass a federal budget in a fight likely to be linked to other issues, such as immigration policy, even as the November congressional election campaigns approach in which Republicans will seek to keep control of Congress. President Donald Trump and his Republicans want a big budget increase in military spending, while Democrats also want proportional increases for non

In [32]:
# Clean the "news" column
# Convert everything to lowercase
df_full["news"] = df_full["news"].apply(lambda x: x.lower())
# Remove punctuation and special characters
df_full["news"] = df_full["news"].apply(lambda x: re.sub(r"[^\w\s]", "", x))
# Remove stopwords
cachedStopWords = stopwords.words("english")
df_full["news"] = df_full["news"].apply(lambda x: " ".join([word for word in x.split() if word not in cachedStopWords]))
# Drop rows where "news" has less than 50 words long
df_full = df_full.loc[df_full["news"].apply(lambda x: len(x.split())) >= 50]

In [33]:
# Display the first news source, after cleaning
df_full.iloc[0]["news"]

'us budget fight looms republicans flip fiscal script washington reuters head conservative republican faction us congress voted month huge expansion national debt pay tax cuts called fiscal conservative sunday urged budget restraint 2018 keeping sharp pivot way among republicans us representative mark meadows speaking cbs face nation drew hard line federal spending lawmakers bracing battle january return holidays wednesday lawmakers begin trying pass federal budget fight likely linked issues immigration policy even november congressional election campaigns approach republicans seek keep control congress president donald trump republicans want big budget increase military spending democrats also want proportional increases nondefense discretionary spending programs support education scientific research infrastructure public health environmental protection trump administration already willing say going increase nondefense discretionary spending 7 percent meadows chairman small influentia

e)	Transform the input text data into feature vectors where the entries of the feature vectors are term-frequency-inverse-document-frequency. Use the TfidfVectorizer() in sklearn. 

In [36]:
# Create the vectorizer
vectorizer = TfidfVectorizer()
# Fit the vectorizer to the news data
news_tfidf = vectorizer.fit_transform(df_full["news"])

f)	Spit the feature vectors and the output variable into into X_train, y_train, X_test and y_test, you can let the test set be 30% of the entire data. 

In [40]:
X_train, X_test, y_train, y_test = train_test_split(news_tfidf, df_full["news_type"], test_size=0.3, random_state=42)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(29153, 230065)
(29153,)
(12495, 230065)
(12495,)


g)	Fit an appropriate Naïve Bayes model and compute the training and test accuracy of the model. Is there overfitting?

In [42]:
# Create a multinomial naive bayes model and fit to the tfidf feature vectors
news_nb = MultinomialNB()
news_nb.fit(X_train, y_train)
# Compute the training accuracy
news_train_preds = news_nb.predict(X_train)
print("Accuracy on the training set:", accuracy_score(y_train, news_train_preds))
# Compute the accuracy on the test set
news_test_preds = news_nb.predict(X_test)
print("Accuracy on the test set:", accuracy_score(y_test, news_test_preds))

Accuracy on the training set: 0.9528350427057249
Accuracy on the test set: 0.9408563425370148


h)	Fit a Naïve Bayes using cross validation and print the average cross validation score as well as the standard deviation of the cross-validation scores.

In [45]:
# Use cross-validation to calculate multiple scores
cv_scores = cross_val_score(news_nb, X_train, y_train)
print("Average cross-validation score:", np.mean(cv_scores))
print("Standard Deviation of cross-validation scores:", np.std(cv_scores))

Average cross-validation score: 0.9413782430394357
Standard Deviation of cross-validation scores: 0.0006981776530629557


i)	Select some hypermeters of your choice and tune using the grid search cross validation. Use some other hyperparameters than those used in class examples. 

In [51]:
# Set parameters of the grid search
param_grid = {"alpha": np.arange(0.05, 1.2, 0.2),
               "fit_prior": [True, False]}
# Fit the gridsearch model
grid = GridSearchCV(MultinomialNB(), param_grid).fit(X_train, y_train)
# Print the best model
print(grid.best_estimator_)

MultinomialNB(alpha=0.05)


In [52]:
# Determine the accuracy of the best model
print(grid.best_score_)

0.9534867257366069


4. 

Mini Project
Find some text data of your own choice, it could be labelled tweets, etc. 
Your dataset should have at least 200 instances, and if there are several columns of text, you can choose to merge the text columns into a single text column. Each text instance should have at least 60 words. 
Clean the data, split the data, transform the data to a representation suitable for your algorithm, build your model and evaluate the model. Tune some parameters of interest and write a short report about what problem your mini project is trying to address, the description of your data, the choice of algorithm used, the performance of your algorithm, overfitting, the choice of hyperparameters tunned, then your recommendation or conclusion (imagine you were trying to recommend this algorithm to a stakeholder, and you need this report to include important and persuasive elements). Your report could be in one or two paragraphs and should include relevant code and output at the end. 


In [82]:
# Load the Spam vs Ham dataset
df = pd.read_csv("spam.csv", encoding='ISO-8859-1')
# Remove unneeded columns
df.drop(["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis=1, inplace=True)
# Rename columns
df.columns = ["label", "text"]
df.head()

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [84]:
# Clean the data
# Remove trailing and leading whitespace, then convert everything to lowercase
df["text"] = df["text"].apply(lambda x: x.strip().lower())
# Remove punctuation and special characters
df["text"] = df["text"].apply(lambda x: re.sub(r"[^\w ]", "", x))
# Remove stopwords
cachedStopWords = stopwords.words("english")
df["text"] = df["text"].apply(lambda x: " ".join([word for word in x.split() if word not in cachedStopWords]))
df.head()

,label,text
0,ham,go jurong point crazy available bugis n great ...
1,ham,ok lar joking wif u oni
2,spam,free entry 2 wkly comp win fa cup final tkts 2...
3,ham,u dun say early hor u c already say
4,ham,nah dont think goes usf lives around though


In [77]:
# Convert the text data to tfidf data
vectorizer = TfidfVectorizer()
text_tfidf = vectorizer.fit_transform(df["text"])

In [78]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(text_tfidf, df["label"], test_size=0.2, random_state=42)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(4457, 9404)
(4457,)
(1115, 9404)
(1115,)


In [79]:
# Fit a Naive Bayes algorithm to the text tfidf data
nb_clf = MultinomialNB()
# Use a grid search to find the optimal parameters
param_grid = {"alpha": np.arange(0.05, 1.2, 0.05)}
grid = GridSearchCV(nb_clf, param_grid).fit(X_train, y_train)
# Find the best model from the gridsearch
best_nb = grid.best_estimator_

In [81]:
# Find the accuracy on the training and test sets
# Training set
train_preds = best_nb.predict(X_train)
print("Accuracy on the training set:", accuracy_score(y_train, train_preds))
# Test set
test_preds = best_nb.predict(X_test)
print("Accuracy on the test set:", accuracy_score(y_test, test_preds))

Accuracy on the training set: 0.9955126766883554
Accuracy on the test set: 0.9838565022421525


In [90]:
# Find the f1 score of the training and test se
print("F1 score on the training set:", f1_score(y_train, train_preds, pos_label="ham"))
print("F1 score on the test set:", f1_score(y_test, test_preds, pos_label="ham"))

F1 score on the training set: 0.9974160206718347
F1 score on the test set: 0.9907407407407408
